![Piggy bank](piggy_bank.jpg)

Personal loans are a lucrative revenue stream for banks. The typical interest rate of a two-year loan in the United Kingdom is [around 10%](https://www.experian.com/blogs/ask-experian/whats-a-good-interest-rate-for-a-personal-loan/). This might not sound like a lot, but in September 2022 alone UK consumers borrowed [around £1.5 billion](https://www.ukfinance.org.uk/system/files/2022-12/Household%20Finance%20Review%202022%20Q3-%20Final.pdf), which would mean approximately £300 million in interest generated by banks over two years!

You have been asked to work with a bank to clean the data they collected as part of a recent marketing campaign, which aimed to get customers to take out a personal loan. They plan to conduct more marketing campaigns going forward so would like you to ensure it conforms to the specific structure and data types that they specify so that they can then use the cleaned data you provide to set up a PostgreSQL database, which will store this campaign's data and allow data from future campaigns to be easily imported. 

They have supplied you with a csv file called `"bank_marketing.csv"`, which you will need to clean, reformat, and split the data, saving three final csv files. Specifically, the three files should have the names and contents as outlined below:

## `client.csv`

| column | data type | description | cleaning requirements |
|--------|-----------|-------------|-----------------------|
| `client_id` | `integer` | Client ID | N/A |
| `age` | `integer` | Client's age in years | N/A |
| `job` | `object` | Client's type of job | Change `"."` to `"_"` |
| `marital` | `object` | Client's marital status | N/A |
| `education` | `object` | Client's level of education | Change `"."` to `"_"` and `"unknown"` to `np.NaN` |
| `credit_default` | `bool` | Whether the client's credit is in default | Convert to `boolean` data type:<br> `1` if `"yes"`, otherwise `0` |
| `mortgage` | `bool` | Whether the client has an existing mortgage (housing loan) | Convert to boolean data type:<br> `1` if `"yes"`, otherwise `0` |

<br>

## `campaign.csv`

| column | data type | description | cleaning requirements |
|--------|-----------|-------------|-----------------------|
| `client_id` | `integer` | Client ID | N/A |
| `number_contacts` | `integer` | Number of contact attempts to the client in the current campaign | N/A |
| `contact_duration` | `integer` | Last contact duration in seconds | N/A |
| `previous_campaign_contacts` | `integer` | Number of contact attempts to the client in the previous campaign | N/A |
| `previous_outcome` | `bool` | Outcome of the previous campaign | Convert to boolean data type:<br> `1` if `"success"`, otherwise `0`. |
| `campaign_outcome` | `bool` | Outcome of the current campaign | Convert to boolean data type:<br> `1` if `"yes"`, otherwise `0`. |
| `last_contact_date` | `datetime` | Last date the client was contacted | Create from a combination of `day`, `month`, and a newly created `year` column (which should have a value of `2022`); <br> **Format =** `"YYYY-MM-DD"` |

<br>

## `economics.csv`

| column | data type | description | cleaning requirements |
|--------|-----------|-------------|-----------------------|
| `client_id` | `integer` | Client ID | N/A |
| `cons_price_idx` | `float` | Consumer price index (monthly indicator) | N/A |
| `euribor_three_months` | `float` | Euro Interbank Offered Rate (euribor) three-month rate (daily indicator) | N/A |

In [10]:
import pandas as pd
import numpy as np

# Start coding here...
df = pd.read_csv("bank_marketing.csv")

client = df.loc[:, ['client_id', 'age', 'job', 'marital', 'education', 'credit_default', 'mortgage']]
campaign = df.loc[:, ['client_id', 'number_contacts', 'contact_duration', 'previous_campaign_contacts', 'previous_outcome', 'campaign_outcome']] # create the last_contact_date
economics = df.loc[:, ['client_id', 'cons_price_idx', 'euribor_three_months']]

client['job'] = client['job'].str.replace(".", "_", regex=False)

# Assert statement to check no full stops remain
assert not client['job'].str.contains(r'\.').any(), "Some values still contain a full stop!"

client['education'] = client['education'].str.replace(".", "_", regex=False)

# Assert statement to check no full stops remain
assert not client['education'].str.contains(r'\.').any(), "Some values still contain a full stop!"

client['education'] = client['education'].replace("unknown", np.nan)

# Assert statement to check no unknown values remain
assert not client['education'].str.contains('unknown').any(), "Some values still contain an unknown value!"

client['credit_default'] = np.where(client['credit_default'] == 'yes', 1, 0)

# Assert statement to check that only 1, or 0 values remain
assert client['credit_default'].isin([0, 1]).all(), "Unexpected values found in 'credit_default' column!"

client['credit_default'] = client['credit_default'].astype('bool')

client['mortgage'] = np.where(client['mortgage'] == 'yes', 1, 0)

# Assert statement to check that only 1, or 0 values remain
assert client['mortgage'].isin([0, 1]).all(), "Unexpected values found in 'mortgage' column!"

client['mortgage'] = client['mortgage'].astype('bool')

campaign['previous_outcome'] = np.where(campaign['previous_outcome'] == 'success', 1, 0)

# Assert statement to check that only 1, or 0 values remain
assert campaign['previous_outcome'].isin([0, 1]).all(), "Unexpected values found in 'previous_outcome' column!"

campaign['previous_outcome'] = campaign['previous_outcome'].astype('bool')

campaign['campaign_outcome'] = np.where(campaign['campaign_outcome'] == 'yes', 1, 0)

# Assert statement to check that only 1, or 0 values remain
assert campaign['campaign_outcome'].isin([0, 1]).all(), "Unexpected values found in 'previous_outcome' column!"

campaign['campaign_outcome'] = campaign['campaign_outcome'].astype('bool')

df['year'] = 2022
df['year'] = df['year'].astype('str')
df['day'] = df['day'].astype('str')
df['date'] = df['year'] + '-' + df['month'] + '-' + df['day']
df['last_contact_date'] = pd.to_datetime(df['date'])

campaign['last_contact_date'] = df['last_contact_date']



client.to_csv('client.csv', index=False)
campaign.to_csv('campaign.csv', index=False)
economics.to_csv('economics.csv', index=False)


    


